In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

import keras
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential 
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import make_scorer

plt.rcParams.update({'font.size': 12})

## CUP MLP

In [ ]:
#importing data
path=r'/home/ludovico/ML-project/data/cup/ML-CUP23-'
train_set = pd.read_csv(path+'TR.csv',skiprows=7, header=None, delimiter=',', dtype=str)

input=train_set[train_set.columns[1:-3]]
target=train_set[train_set.columns[-3:]]

#splitting design set from test set (test set will be used only for the final model assessment)
#the random seed is fixed to use the same design set for all the models

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=0, shuffle=True)

x_train=x_train.astype(np.float64)
y_train=y_train.astype(np.float64)

#we add this metric (Mean euclidean error) to evaluate the performance of the model 
def MEE(x, y):
    return np.mean(np.linalg.norm(x - y, 2, axis=1))

## Creating network with 1 layer using Keras 
We add 1 hidden layer with 10 (x_train.shape[1]) number of input units, dens_nparams number of units in the hidden layer, SGD as optimizer algorithm and 3 output unit with linear activation function. 

In [ ]:

def create_model_1(init='uniform', 
                activation='relu',
                nbr_features=10, 
                dense_nparams=10, 
                lr_init=0.1, 
                momentum=0.1, 
                w_d=0.0001, 
                nest=False,
                decay_steps=100,
                decay_rate=0.9):
    
    model = Sequential()
    model.add(Dense(dense_nparams, activation, input_shape=(nbr_features,), kernel_initializer=init,)) 
    model.add(Dense(3, activation='linear', kernel_initializer=init))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    lr_init,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

    model.compile(loss='mean_squared_error', optimizer=SGD(lr_schedule,momentum,nest,w_d))
    return model

#callback = keras.callbacks.EarlyStopping(monitor='loss',patience=3)
keras_estimator = KerasRegressor(model=create_model_1,verbose=0)

## Grid search for 1 layer

In [ ]:
keras.utils.set_random_seed(42)


param_grid = {
    'epochs': [1500],
    'model__dense_nparams': [100],
    'model__init': [ 'uniform' ], 
    'batch_size':[50],
    'model__lr_init':[0.02],
    'model__decay_steps':[1000],
    'model__decay_rate':[0.90],
    'model__momentum':[0.5],
    'model__w_d':[0.001],
    'model__activation':['tanh'],
    'model__nest':[False]    
}

grid_search_1 = GridSearchCV(
    estimator=keras_estimator,
    param_grid=param_grid,
    cv=RepeatedKFold(n_splits=5, n_repeats=5, random_state=0),
    n_jobs=-1,
    return_train_score = True,
    refit=True,
    scoring=make_scorer(MEE, greater_is_better=False),
    verbose=3,
    error_score='raise'
)

MLP_1=grid_search_1.fit(x_train, y_train)


## Evaluating the model

In [ ]:
cv_results_df = pd.DataFrame(MLP_1.cv_results_)
best_model_index=MLP_1.best_index_

print('best params',MLP_1.best_params_) 

val_loss=cv_results_df['mean_test_score'][best_model_index]
val_std=cv_results_df['std_test_score'][best_model_index]
train_loss=cv_results_df['mean_train_score'][best_model_index]
train_std=cv_results_df['std_train_score'][best_model_index]
print('Train loss:',train_loss,'+/-', train_std)
print('Validation loss:',val_loss,'+/-', val_std)

cv_results_df

## Learning curve

In [ ]:
keras.utils.set_random_seed(42)

# Splitting the design set
x_train_cl, x_val, y_train_cl, y_val = train_test_split(x_train, y_train, test_size=0.20, shuffle=True)

model=KerasRegressor(model=create_model_1,**grid_search_1.best_params_)

history = model.fit(x_train_cl, y_train_cl.astype(np.float64),validation_data=(x_val,y_val.astype(np.float64)))

print(history.history_.keys())

# summarize history for loss
plt.plot(history.history_['loss'])
plt.plot(history.history_['val_loss'], linestyle='--')

plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.yscale('log')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

## Creating network with 2 layers using Keras 
We add 2 hidden layers with 10 (x_train.shape[1]) number of input units, dens_nparams number of units in the hidden layer, SGD as optimizer algorithm and 3 output unit with linear activation function. 

In [ ]:
def create_model_2(init='uniform', 
                activation='relu', 
                nbr_features=10, 
                dense_nparams=10,
                lr_init=0.1,
                momentum=0.1,
                w_d=0.0001, 
                nest=False,
                decay_steps=100,
                decay_rate=0.9):
    
    model = Sequential()
    model.add(Dense(dense_nparams, activation, input_shape=(nbr_features,), kernel_initializer=init,)) 
    model.add(Dense(dense_nparams, activation, kernel_initializer=init))
    model.add(Dense(3, activation='linear', kernel_initializer=init))

    
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    lr_init,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

    model.compile(loss='mean_squared_error', optimizer=SGD(lr_schedule,momentum,nest,w_d))
    return model

keras_estimator = KerasRegressor(model=create_model_2,verbose=0)

## Grid search for 2 layers

In [ ]:
keras.utils.set_random_seed(42)

param_grid = {
    'epochs': [3000],
    'model__dense_nparams': [110],
    'model__init': [ 'uniform' ], 
    'batch_size':[200],
    'model__lr':[0.01],
    'model__momentum':[0.8],
    'model__w_d':[0.001],
    'model__activation':['tanh'],
    'model__nest':[True],
    'model__decay_steps':[500]   
}

grid_search_2 = GridSearchCV(
    estimator=keras_estimator,
    param_grid=param_grid,
    cv=RepeatedKFold(n_splits=5, n_repeats=5, random_state=0),
    n_jobs=-1,
    return_train_score = True,
    refit=True,
    scoring=make_scorer(MEE, greater_is_better=False),
    verbose=3,
    error_score='raise'
)

MLP_2=grid_search_2.fit(x_train, y_train)


## Evaluating the model

In [ ]:
cv_results_df = pd.DataFrame(MLP_2.cv_results_)
best_model_index=MLP_2.best_index_

print('best params',MLP_2.best_params_) 

val_loss=cv_results_df['mean_test_score'][best_model_index]
val_std=cv_results_df['std_test_score'][best_model_index]
train_loss=cv_results_df['mean_train_score'][best_model_index]
train_std=cv_results_df['std_train_score'][best_model_index]
print('Train loss:',train_loss,'+/-', train_std)
print('Validation loss:',val_loss,'+/-', val_std)

cv_results_df

## Learning curve

In [ ]:
keras.utils.set_random_seed(42)

# Splitting the design set
x_train_cl, x_val, y_train_cl, y_val = train_test_split(x_train, y_train, test_size=0.20, shuffle=True)

model=KerasRegressor(model=create_model_2,**grid_search_2.best_params_)

history = model.fit(x_train_cl, y_train_cl.astype(np.float64),validation_data=(x_val,y_val.astype(np.float64)))

print(history.history_.keys())

# summarize history for loss
plt.plot(history.history_['loss'])
plt.plot(history.history_['val_loss'], linestyle='--')

plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.yscale('log')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

## The best model between 1 layer and 2 layers is manually selected as the one with the lowest MEE on validation